In [14]:
import keras
keras.__version__

'2.2.4'

# Classifying traffic conditions or hazards: a multi-class classification example


In [15]:
import os, shutil, random

In [16]:
# The path to the directory where the original
# dataset was uncompressed
original_dataset_dir = r'./trafficnet_dataset_v1'


train_dir = os.path.join(original_dataset_dir, 'train')
validation_dir = os.path.join(original_dataset_dir, 'validation')
test_dir = os.path.join(original_dataset_dir, 'test')
os.mkdir(validation_dir)

accident_train_dir = os.path.join(train_dir, 'accident')
dense_traffic_train_dir = os.path.join(train_dir, 'dense_traffic')
fire_train_dir = os.path.join(train_dir, 'fire')
sparse_traffic_train_dir = os.path.join(train_dir, 'sparse_traffic')

accident_test_dir = os.path.join(test_dir, 'accident')
dense_traffic_test_dir = os.path.join(test_dir, 'dense_traffic')
fire_test_dir = os.path.join(test_dir, 'fire')
sparse_traffic_test_dir = os.path.join(test_dir, 'sparse_traffic')

# Directory with our validation accident pictures
validation_accident_dir = os.path.join(validation_dir, 'accident')
os.mkdir(validation_accident_dir)
validation_dense_traffic_dir = os.path.join(validation_dir, 'dense_traffic')
os.mkdir(validation_dense_traffic_dir)
validation_fire_dir = os.path.join(validation_dir, 'fire')
os.mkdir(validation_fire_dir)
validation_sparse_traffic_dir = os.path.join(validation_dir, 'sparse_traffic')
os.mkdir(validation_sparse_traffic_dir)



In [17]:
print('total training accident images:', len(os.listdir(accident_train_dir)))
print('total training dense traffic images:', len(os.listdir(dense_traffic_train_dir)))
print('total training fire images:', len(os.listdir(fire_train_dir)))
print('total training sparse traffic images:', len(os.listdir(sparse_traffic_train_dir)))
print("\n")
print('total test accident images:', len(os.listdir(accident_test_dir)))
print('total test dense traffic images:', len(os.listdir(dense_traffic_test_dir)))
print('total test fire images:', len(os.listdir(fire_test_dir)))
print('total test sparse traffic images:', len(os.listdir(sparse_traffic_test_dir)))

total training accident images: 900
total training dense traffic images: 900
total training fire images: 900
total training sparse traffic images: 901


total test accident images: 200
total test dense traffic images: 200
total test fire images: 200
total test sparse traffic images: 200


In [18]:
#Taking a 80:20 split of training and validation data

validation_accident_len = int(len(os.listdir(accident_train_dir)) * 0.2)
print('validation_accident_len', validation_accident_len)

validation_dense_traffic_len = int(len(os.listdir(dense_traffic_train_dir)) * 0.2)
print('validation_dense_traffic_len', validation_dense_traffic_len)
validation_fire_len = int(len(os.listdir(fire_train_dir)) * 0.2)
print('validation_fire_len', validation_fire_len)
validation_sparse_traffic_len = int(len(os.listdir(sparse_traffic_train_dir)) * 0.2)
print('validation_sparse_traffic_len', validation_sparse_traffic_len)




validation_accident_len 180
validation_dense_traffic_len 180
validation_fire_len 180
validation_sparse_traffic_len 180


In [19]:
# Adding images to validation directory
training_accident_files = os.listdir(accident_train_dir)
training_dense_traffic_files = os.listdir(dense_traffic_train_dir)
training_fire_files = os.listdir(fire_train_dir)
training_sparse_traffic_files = os.listdir(sparse_traffic_train_dir)
#print( random.sample(training_accident_files,validation_accident_len ))


for accident_img_file in random.sample(training_accident_files,validation_accident_len ):
    srcFile = os.path.join(accident_train_dir,  accident_img_file)
    shutil.move(srcFile,validation_accident_dir )
    
for dense_traffic_img_file in random.sample(training_dense_traffic_files,validation_dense_traffic_len ):
    srcFile = os.path.join(dense_traffic_train_dir,  dense_traffic_img_file)
    shutil.move(srcFile,validation_dense_traffic_dir )
    
for fire_img_file in random.sample(training_fire_files,validation_fire_len ):
    srcFile = os.path.join(fire_train_dir ,  fire_img_file)
    shutil.move(srcFile,validation_fire_dir )
    
for sparse_traffic_img_file in random.sample(training_sparse_traffic_files,validation_sparse_traffic_len ):
    srcFile = os.path.join(sparse_traffic_train_dir,  sparse_traffic_img_file)
    shutil.move(srcFile,validation_sparse_traffic_dir )  


In [33]:
print('total validation accident images:', len(os.listdir(validation_accident_dir)))
print('total validation dense traffic images:', len(os.listdir(validation_dense_traffic_dir)))
print('total validation fire images:', len(os.listdir(validation_fire_dir)))
print('total validation sparse traffic images:', len(os.listdir(validation_sparse_traffic_dir)))

total validation accident images: 180
total validation dense traffic images: 180
total validation fire images: 180
total validation sparse traffic images: 180


## Building our network


In [34]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='tanh',
                        input_shape=(200, 100, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='tanh'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='tanh'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='tanh'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='tanh'))
model.add(layers.Dense(4, activation='softmax'))

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 198, 98, 32)       896       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 99, 49, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 97, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 48, 23, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 46, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 23, 10, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 21, 8, 128)        147584    
__________

Compilation, loss function and optimiser selection

In [36]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Data preprocessing


In [37]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True)

validate_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(200, 100),
        batch_size=30,
        color_mode='rgb',
        class_mode='categorical')

validation_generator = validate_datagen.flow_from_directory(
        validation_dir,
        target_size=(200, 100),
        batch_size=30,
        color_mode='rgb',
        class_mode='categorical')

Found 2881 images belonging to 4 classes.
Found 719 images belonging to 4 classes.


In [38]:

for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (30, 200, 100, 3)
labels batch shape: (30, 4)


## Model Fitting

In [39]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=96,
      epochs=30,
      validation_data=validation_generator,
      use_multiprocessing=True,
      validation_steps=24)

Epoch 1/30
96/96 [==============================] - 46s 480ms/step - loss: 0.9812 - acc: 0.5927 - val_loss: 1.4689 - val_acc: 0.4673
Epoch 2/30
96/96 [==============================] - 46s 476ms/step - loss: 0.8235 - acc: 0.6708 - val_loss: 0.7818 - val_acc: 0.6857
Epoch 3/30
96/96 [==============================] - 45s 473ms/step - loss: 0.7375 - acc: 0.7218 - val_loss: 0.6679 - val_acc: 0.7413
Epoch 4/30
96/96 [==============================] - 45s 472ms/step - loss: 0.7358 - acc: 0.7140 - val_loss: 0.6465 - val_acc: 0.7538
Epoch 5/30
96/96 [==============================] - 45s 473ms/step - loss: 0.6743 - acc: 0.7475 - val_loss: 0.6722 - val_acc: 0.7302
Epoch 6/30
96/96 [==============================] - 45s 472ms/step - loss: 0.6561 - acc: 0.7493 - val_loss: 0.6403 - val_acc: 0.7622
Epoch 7/30
96/96 [==============================] - 45s 472ms/step - loss: 0.6427 - acc: 0.7548 - val_loss: 0.6787 - val_acc: 0.7650
Epoch 8/30
96/96 [==============================] - 45s 471ms/step - 

In [40]:
model.save('PredictingTrafficConditionsHazards_i150_b30_3cl_i2.h5')

## Plotting Training and validation Accuracy and loss

In [41]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [42]:
tester_datagen = ImageDataGenerator(rescale=1./255)

test_generator = tester_datagen.flow_from_directory(
        test_dir,
        target_size=(200, 100),
        batch_size=1,
        class_mode='categorical',
        shuffle = 'False')

results = model.evaluate_generator(test_generator,workers=6,steps=800 )

Found 800 images belonging to 4 classes.


In [43]:
print(results)

[0.671826478245473, 0.75625]
